In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Groupby apply on large (relational) data set

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-08-5')].copy()

In [4]:
test_data

,date,state,country,confirmed
52444,2020-08-06,no,Germany,215039.0
52485,2020-08-06,no,US,4883582.0
52710,2020-08-07,no,Germany,216196.0
52750,2020-08-07,no,US,4941755.0
52977,2020-08-08,no,Germany,216903.0
53017,2020-08-08,no,US,4997929.0
53243,2020-08-09,no,Germany,217288.0
53283,2020-08-09,no,US,5044864.0
53508,2020-08-10,no,Germany,218508.0
53548,2020-08-10,no,US,5094400.0


In [5]:
test_data.groupby(['country']).agg(np.max)


,date,state,confirmed
country,,,
Germany,2020-08-15,no,224488.0
US,2020-08-15,no,5361165.0


In [6]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    #print(in_array)
    #print(type(in_array))
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [7]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [8]:
test_data.groupby(['state','country']).apply(rolling_reg)

state  country       
no     Germany  52444           NaN
                52710           NaN
                52977    231.809013
                53243    397.061661
                53508    271.110696
                53775    194.001184
                54041    186.844463
                54306    161.177186
                54573    151.644156
                54838    202.555505
       US       52485           NaN
                52750           NaN
                53017     86.422707
                53283     96.884837
                53548    104.606172
                53815    105.735503
                54081     99.879424
                54361     96.442858
                54614     90.696852
                54880     94.607140
Name: confirmed, dtype: float64

In [9]:
test_data

,date,state,country,confirmed
52444,2020-08-06,no,Germany,215039.0
52485,2020-08-06,no,US,4883582.0
52710,2020-08-07,no,Germany,216196.0
52750,2020-08-07,no,US,4941755.0
52977,2020-08-08,no,Germany,216903.0
53017,2020-08-08,no,US,4997929.0
53243,2020-08-09,no,Germany,217288.0
53283,2020-08-09,no,US,5044864.0
53508,2020-08-10,no,Germany,218508.0
53548,2020-08-10,no,US,5094400.0


In [10]:
pd_DR_result=pd_JH_data.groupby(['state','country']).apply(rolling_reg).reset_index()

In [11]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR','level_2':'index'})

In [12]:
pd_DR_result

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1197,NaN
...,...,...,...,...
55057,no,Zimbabwe,53864,56.074951
55058,no,Zimbabwe,54062,66.478161
55059,no,Zimbabwe,54359,56.980620
55060,no,Zimbabwe,54594,55.698324


In [13]:
pd_JH_data=pd_JH_data.reset_index()

In [14]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN
...,...,...,...,...,...,...
55057,55057,2020-08-15,no,Barbados,150.0,49.111111
55058,55058,2020-08-15,no,Belarus,69308.0,1319.485714
55059,55059,2020-08-15,no,Belgium,77869.0,91.844656
55060,55060,2020-08-15,no,Albania,7260.0,49.245675


# FIltering Data With Groupby Application

In [15]:
from scipy import signal
def  savgol_filter(df_input, column='confirmed',window=5):
    window=5
    degree=1
    df_result=df_input
    filter_in=df_input[column].fillna(0)
    result=signal.savgol_filter(np.array(filter_in),window,degree)
    df_result[column+'_filtered']=result
    return df_result


In [16]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [17]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


# Filtered Doubling Rate

In [18]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()
pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR','level_2':'index'})
pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
55057,no,Zimbabwe,53864,55.248541
55058,no,Zimbabwe,54062,57.519491
55059,no,Zimbabwe,54359,57.632589
55060,no,Zimbabwe,54594,57.008567
55061,no,Zimbabwe,55061,56.751955


In [19]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
55057,55057,2020-08-15,no,Barbados,150.0,49.111111,149.4,82.000000
55058,55058,2020-08-15,no,Belarus,69308.0,1319.485714,69347.6,853.034483
55059,55059,2020-08-15,no,Belgium,77869.0,91.844656,77803.2,107.240401
55060,55060,2020-08-15,no,Albania,7260.0,49.245675,7261.8,48.467302
55061,55061,2020-08-15,no,Zimbabwe,5176.0,54.616487,5168.8,56.751955


In [20]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask,other=np.NaN)

In [21]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
53775,53775,2020-08-11,no,Germany,219540.0,194.001184,219695.2,217.774392
54041,54041,2020-08-12,no,Germany,220859.0,186.844463,220995.8,184.976180
54306,54306,2020-08-13,no,Germany,222281.0,161.177186,222191.8,177.009480
54573,54573,2020-08-14,no,Germany,223791.0,151.644156,223474.6,179.297025
54838,54838,2020-08-15,no,Germany,224488.0,202.555505,224757.4,174.208450


In [22]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)